<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Hyperparameter_Optimization_(HPO).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install optuna

In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.nn.functional as F  # Import required functional module
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define a simple model
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))  # Apply relu from nn.functional
        x = self.fc2(x)
        return x

# Objective function for Optuna
def objective(trial):
    hidden_dim = trial.suggest_int('hidden_dim', 32, 128)
    learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-1)

    model = SimpleNN(input_dim=10, hidden_dim=hidden_dim, output_dim=2)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Dummy data (train and validation)
    x_train = torch.randn(100, 10)
    y_train = torch.randint(0, 2, (100,))
    train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=16, shuffle=True)

    x_val = torch.randn(30, 10)
    y_val = torch.randint(0, 2, (30,))
    val_loader = DataLoader(TensorDataset(x_val, y_val), batch_size=16, shuffle=False)

    # Training loop
    for epoch in range(10):
        model.train()
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

    # Validation loop
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for x_batch, y_batch in val_loader:
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            total_val_loss += loss.item()

    # Return average validation loss
    return total_val_loss / len(val_loader)

# Running Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# Output best trial results
print(f"Best trial parameters: {study.best_params}")
print(f"Best validation loss: {study.best_value}")